In [1]:
%run ../../import_src.py

import lymedata
from lymedata import *
import constants
from constants import *

import tensorflow as tf
from tensorflow import keras
from keras import layers

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

2025-03-02 15:31:50.171580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-02 15:31:50.183812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-02 15:31:50.187447: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 15:31:50.196315: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
tf.keras.backend.clear_session()

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

data = LymeData({CHRONIC, NEURO, MUSCULO},{ADDL_CIR},{NEURO, MUSCULO, BOTH, NEITHER}, defn=DEF_CNS1, drop_99=True)
print(data.df.shape)
print(data.df.columns)

X, Y = data.get_data_and_labels()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

for arr in [X_train, X_test, Y_train, Y_test]:
    print(arr.shape)
# Results of shape:
# (2878, 12)
# (720, 12)
# (2878, 4)
# (720, 4)
num_train_labels = X_train.shape[1]

# automatically one-hot-encoded
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(num_train_labels, )),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,Y_train, epochs=50, batch_size=32) 

# 3 layers:
# 30 epochs: 63.1% train, 62.0 test
# 35 epochs: 0.6206 train, 0.6296 test
# 40 epochs: 0.6076 train, 0.6173

# 4 layers:
# 50 epochs: 63.3% train, 61.4% test

pred = model.predict(X_test)

Y_pred = np.argmax(pred, axis=1)
Y_true = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(Y_true, Y_pred)
print(Y_pred)
print(f'Accuracy: {accuracy:.4f}')


/home/reu24lyme/alex/code_lib/src/lymedata.py:74: DtypeWarning: Columns (224,249,259,269,274,294,299,304,309,339,344,365,369,373,377,395,399,403,411,415,419,423,435) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Both Neuro and Mus: 2259
Only Neuro: 310
Only Mus: 762
Neither Neuro nor Mus: 632
Dropping 99
(1619, 12)
Index(['Bio Sex', 'Antibiotics', 'Times Infected', 'GROC', 'Bed Days',
       'Mental Health Days', 'Physical Health Days', 'Disability', 'both',
       'neither', 'musculo', 'neuro'],
      dtype='object')
(1295, 8)
(324, 8)
(1295, 4)
(324, 4)
Epoch 1/50


/home/reu24lyme/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.5135 - loss: 1.2083
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6122 - loss: 1.0003
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6003 - loss: 0.9876
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6243 - loss: 0.9386
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6370 - loss: 0.9019
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5981 - loss: 0.9443
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6183 - loss: 0.9296
Epoch 8/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6317 - loss: 0.9392
Epoch 9/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6091 - loss: 0.9464
Epoch 10/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6280 - loss: 0.8877
Epoch 11/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6055 - loss: 0.9333
Epoch 12/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6484 - loss: 0.9107

In [8]:
tf.keras.backend.clear_session()

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

data = LymeData({CHRONIC, NEURO, MUSCULO},{ADDL_CIR, DIAG_CIR},{NEURO, MUSCULO, BOTH, NEITHER}, defn=DEF_CNS1, drop_99=True)
print(data.df.shape)
print(data.df.columns)

X, Y = data.get_data_and_labels()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

for arr in [X_train, X_test, Y_train, Y_test]:
    print(arr.shape)
# Results of shape:
# (2878, 12)
# (720, 12)
# (2878, 4)
# (720, 4)
num_train_labels = X_train.shape[1]

# automatically one-hot-encoded
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(num_train_labels, )),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,Y_train, epochs=30, batch_size=32) 

# 3 layers:
# 30 epochs: 63.1% train, 62.0 test
# 35 epochs: 0.6206 train, 0.6296 test
# 40 epochs: 0.6076 train, 0.6173

# 4 layers:
# 50 epochs: 63.3% train, 61.4% test

pred = model.predict(X_test)

Y_pred = np.argmax(pred, axis=1)
Y_true = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(Y_true, Y_pred)
print(Y_pred)
print(f'Accuracy: {accuracy:.4f}')


/home/reu24lyme/alex/code_lib/src/lymedata.py:74: DtypeWarning: Columns (224,249,259,269,274,294,299,304,309,339,344,365,369,373,377,395,399,403,411,415,419,423,435) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Both Neuro and Mus: 2259
Only Neuro: 310
Only Mus: 762
Neither Neuro nor Mus: 632
Dropping 99
(886, 24)
Index(['recall a tick bite', 'length of time noticed tick bite',
       'treated with antibiotics', 'length of time treated for tick bite',
       'period of time for diagnosis', 'misdiagnosis', 'tick born coinfection',
       'Babesia', 'Bartonella', 'Ehrlichia/ Anaplasma', 'Mycoplasma',
       'Rickettsia', 'Bio Sex', 'Antibiotics', 'Times Infected', 'GROC',
       'Bed Days', 'Mental Health Days', 'Physical Health Days', 'Disability',
       'both', 'neither', 'musculo', 'neuro'],
      dtype='object')
(708, 20)
(178, 20)
(708, 4)
(178, 4)
Epoch 1/30


/home/reu24lyme/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.5136 - loss: 1.2294
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5579 - loss: 1.1064
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6156 - loss: 0.9775
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5869 - loss: 0.9968
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5964 - loss: 0.9945
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6297 - loss: 0.9642
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6065 - loss: 0.9660
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6128 - loss: 0.9895
Epoch 9/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6399 - loss: 0.9118
Epoch 10/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6365 - loss: 0.9378
Epoch 11/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6270 - loss: 0.9272
Epoch 12/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6404 - loss: 0.9075

In [9]:
tf.keras.backend.clear_session()

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

data = LymeData({CHRONIC, NEURO, MUSCULO},{CATG},{NEURO, MUSCULO, BOTH, NEITHER}, defn=DEF_CNS1, drop_99=True)
print(data.df.shape)
print(data.df.columns)

X, Y = data.get_data_and_labels()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

for arr in [X_train, X_test, Y_train, Y_test]:
    print(arr.shape)
# Results of shape:
# (2878, 12)
# (720, 12)
# (2878, 4)
# (720, 4)
num_train_labels = X_train.shape[1]

# automatically one-hot-encoded
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(num_train_labels, )),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,Y_train, epochs=50, batch_size=32) 

# 3 layers:
# 30 epochs: 63.1% train, 62.0 test
# 35 epochs: 0.6206 train, 0.6296 test
# 40 epochs: 0.6076 train, 0.6173

# 4 layers:
# 50 epochs: 63.3% train, 61.4% test

pred = model.predict(X_test)

Y_pred = np.argmax(pred, axis=1)
Y_true = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(Y_true, Y_pred)
print(Y_pred)
print(f'Accuracy: {accuracy:.4f}')


/home/reu24lyme/alex/code_lib/src/lymedata.py:74: DtypeWarning: Columns (224,249,259,269,274,294,299,304,309,339,344,365,369,373,377,395,399,403,411,415,419,423,435) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Both Neuro and Mus: 2259
Only Neuro: 310
Only Mus: 762
Neither Neuro nor Mus: 632
Dropping 99
(3927, 11)
Index(['general practitioner', 'infectious disease specialist', 'internist',
       'lyme specialist', 'other doctor', 'pediatrician', 'rheumatologist',
       'both', 'neither', 'musculo', 'neuro'],
      dtype='object')
(3141, 7)
(786, 7)
(3141, 4)
(786, 4)
Epoch 1/50


/home/reu24lyme/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


99/99 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5266 - loss: 1.2224
Epoch 2/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5524 - loss: 1.1584
Epoch 3/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5571 - loss: 1.1479
Epoch 4/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5693 - loss: 1.1319
Epoch 5/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5602 - loss: 1.1411
Epoch 6/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5624 - loss: 1.1400
Epoch 7/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5693 - loss: 1.1217
Epoch 8/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5667 - loss: 1.1337
Epoch 9/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5662 - loss: 1.1281
Epoch 10/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5710 - loss: 1.1364
Epoch 11/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5679 - loss: 1.1258
Epoch 12/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5679 - loss: 1.1339